In [1]:
import requests
import time
import json
import numpy as np
import blockchain
import BusClass

In [2]:
url = "https://api.thingspeak.com/channels/1704499/feeds.json?api_key=xxxx&results=2"
Rkey = "xxxx"
latStation = [7.865661,7.881308,7.893300,7.887864,7.890755,7.882510,7.885414] # lat of stantion point
lonStation = [98.397750,98.364982,98.368747,98.371470,98.390563,98.404646,98.430988] # lon of stantion point
enitalB1 = 0
enitalB2 = 0

In [3]:
def getBusStopPoint() :

    resp = requests.get(url)
    data_disc = json.loads(resp.text)

    stationId = data_disc["feeds"][1]["field4"]
    point = []

    if stationId == "1.0" :
        point = [latStation[0],lonStation[0]]
    
    elif stationId == "2.0" :
        point = [latStation[1],lonStation[1]]
    
    elif stationId == "3.0" :
        point = [latStation[2],lonStation[2]]
    
    elif stationId == "4.0" :
        point = [latStation[3],lonStation[3]]
    
    elif stationId == "5.0" :
        point = [latStation[4],lonStation[4]]
    
    elif stationId == "6.0" :
        point = [latStation[5],lonStation[5]]
    
    elif stationId == "7.0" :
        point = [latStation[6],lonStation[6]]
        
    else:
        point = ["X","X"]
        
    return point


In [4]:
def addData(data):
    baseUrlWr = "https://api.thingspeak.com/update?api_key="
    urlWr = baseUrlWr + Rkey
    dataAdd = ""
    for i in range(len(data)):
        dataAdd = dataAdd + "&field"+ str(i+1) +"=" + str(data[i])
        
    urlWr = baseUrlWr + Rkey + dataAdd

    requests.get(urlWr)
    

In [5]:
bc = blockchain.Blockchain()# create first blockchain

distanceFormGoal = 0
speed = 0
estimateTimeToBusStop = 0

In [6]:
while (bc.is_chain_valid()):# check validation of blockchain
    resp = requests.get(url)
    data_disc = json.loads(resp.text)
    busID = data_disc["feeds"][1]["field5"]
    stationID = data_disc["feeds"][1]["field4"]
    busStopPoint = getBusStopPoint()
    
    if(busStopPoint == ["X","X"]):
        print("Station ID is invalid (1 - 7)")
        break
        
    if(not ((busID == '1.0')|(busID == '2.0'))):
        print("Bus ID is invalid (1 - 2)")
        break
    
    if((busID == '1.0') & (enitalB1 == 0)):

        busId1 = BusClass.Bus(float(data_disc["feeds"][1]["field1"]),float(data_disc["feeds"][1]["field2"])) 
        distanceFormGoal = busId1.distanceFormGoal(busStopPoint[0],busStopPoint[1])
        speed = busId1.readSpeed()
        enitalB1 = 1;
        
    if((busID == '2.0') & (enitalB2 == 0)):

        busId2 = BusClass.Bus(float(data_disc["feeds"][1]["field1"]),float(data_disc["feeds"][1]["field2"])) 
        distanceFormGoal = busId2.distanceFormGoal(busStopPoint[0],busStopPoint[1])
        speed = busId2.readSpeed()
        enitalB2 = 1;
        
    if((busID == '1.0') & (enitalB1 > 1)):
    
        busId1.setBusLocation(float(data_disc["feeds"][1]["field1"]),float(data_disc["feeds"][1]["field2"]))
        distanceFormGoal = busId1.distanceFormGoal(busStopPoint[0],busStopPoint[1])
        speed = busId1.readSpeed()
        
    if((busID == '2.0') & (enitalB2 > 1)):

        busId2.setBusLocation(float(data_disc["feeds"][1]["field1"]),float(data_disc["feeds"][1]["field2"]))
        distanceFormGoal = busId2.distanceFormGoal(busStopPoint[0],busStopPoint[1])
        speed = busId2.readSpeed()
        
    estimateTimeToBusStop = (distanceFormGoal/speed) * 60
   
    if((busID == '1.0') & (enitalB1 == 1)):
        
        enitalB1 = 2
        
    if((busID == '2.0') & (enitalB2 == 1)):
        
        enitalB2 = 2
        
    bc.mine_block("Bus ID :" + str(busID) + 
                  ",Station ID :" + str(stationID) + 
                  ",Distance to bus stop :" + str(distanceFormGoal) +
                  ",Speed :" + str(speed) + 
                  ",Estimate time to bus stop :"+ str(estimateTimeToBusStop)
                 )
    dataForAdd = [bc.chain[len(bc.chain)-1]["index"],
                 bc.is_chain_valid(),
                 busID,
                 stationID,
                 distanceFormGoal,
                 speed,
                 estimateTimeToBusStop,
                 bc.chain[len(bc.chain) - 1]["previous_hash"]]
    
    addData(dataForAdd)
                  
    print(bc.chain[len(bc.chain)-1])# print the current block
    time.sleep(25)

C:\Users\ziver\AppData\Local\Temp\ipykernel_18676\4023968071.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  estimateTimeToBusStop = (distanceFormGoal/speed) * 60


{'index': 2, 'timestamp': '2022-10-29 22:27:42.685096', 'data': 'Bus ID :2.0,Station ID :7.0,Distance to bus stop :7.356502336531929,Speed :0.0,Estimate time to bus stop :inf', 'proof': 8111, 'previous_hash': '48fc3b4704e12f94d6ac3341150fd76c62b7651910092084c2f84e6a87739253'}
{'index': 3, 'timestamp': '2022-10-29 22:28:11.182606', 'data': 'Bus ID :2.0,Station ID :7.0,Distance to bus stop :7.356502336531929,Speed :0.0,Estimate time to bus stop :inf', 'proof': 2244, 'previous_hash': '5c0e4a5d182aed8047ddc8b6f102697615cc5e3f5a450ca9e24236233ad06185'}
{'index': 4, 'timestamp': '2022-10-29 22:28:39.790233', 'data': 'Bus ID :2.0,Station ID :7.0,Distance to bus stop :7.356502336531929,Speed :0.0,Estimate time to bus stop :inf', 'proof': 579, 'previous_hash': '62bf98bd05975a2b14f965cb73b630fe9708c660187ae9e867945bee4f2289e4'}
{'index': 5, 'timestamp': '2022-10-29 22:29:08.326593', 'data': 'Bus ID :1.0,Station ID :1.0,Distance to bus stop :8.5515314081372,Speed :0.0,Estimate time to bus stop :i

KeyboardInterrupt: 